#Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Clone WMT22-MasaKhane GitHub Repo

In [1]:
%cd /content/drive/MyDrive

!git clone https://github.com/abumafrim/WMT22-MasaKhane.git

/content/drive/MyDrive
Cloning into 'WMT22-MasaKhane'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 120 (delta 58), reused 66 (delta 20), pack-reused 0
Receiving objects: 100% (120/120), 2.15 MiB | 9.43 MiB/s, done.
Resolving deltas: 100% (58/58), done.


#Set-up

*   Change directory to WMT22-MasaKhane
*   Install required libraries

In [1]:
%cd /content/drive/MyDrive/WMT22-MasaKhane

!pip install -r requirements.txt

/content/drive/MyDrive/WMT22-MasaKhane
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 884 kB 5.2 MB/s 
     |████████████████████████████████| 1.8 MB 71.2 MB/s 
     |████████████████████████████████| 40 kB 7.1 MB/s 
     |████████████████████████████████| 3.0 MB 66.3 MB/s 
     |████████████████████████████████| 1.2 MB 51.4 MB/s 
     |████████████████████████████████| 880 kB 80.1 MB/s 
     |████████████████████████████████| 212 kB 78.6 MB/s 
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=670dbdd38aacb6b51b3fd44aba8b4b52d27f741ec9e1342512a1fffa3d6b520a
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=7d55c25a8bdb43dd64053e5306fb4225c368ab8493177c2485473ae946aad90f
  Stored in directory: /root/.cache/pip/whe

#Download data

* MAFAND-MT
* Huggingface LASER
* Create Sentence-pair classification dataset

In [7]:
%cd /content/drive/MyDrive/WMT22-MasaKhane

#Clone mafand and preprocess data files
!git clone https://github.com/masakhane-io/lafand-mt.git
!python3 download-and-process-mafand.py

#Download Hugginface LASER and preprocess data files
!python3 download-and-process-hug-laser.py

%cd sentence-pair-classification
!python3 create-spc-data.py

/content/drive/MyDrive/WMT22-MasaKhane
/content/drive/MyDrive/WMT22-MasaKhane/sentence-pair-classification
Creation of the eng-lug folder...
Finished: eng-lug
Creation of the eng-tsn folder...
Finished: eng-tsn
Creation of the eng-zul folder...
Finished: eng-zul
Creation of the fra-wol folder...
Finished: fra-wol


#Train SPC Model

In [ ]:
%cd /content/drive/MyDrive/WMT22-MasaKhane/sentence-pair-classification

import os

##Provide the model to finetune
## Any of "albert-base-v2", "albert-large-v2", "albert-xlarge-v2", "albert-xxlarge-v2", "bert-base-uncased", etc

model = 'albert-base-v2'

#hug_langs = ['eng-hau', 'eng-ibo', 'eng-lug', 'eng-swh', 'eng-tsn', 'eng-yor', 'eng-zul', 'fra-wol']
#maf_langs = ['en_hau', 'en_ibo', 'en_lug', 'en_swa', 'en_tsn', 'en_yor', 'en_zul', 'fr_wol']

hug_langs = ['eng-lug', 'eng-tsn', 'eng-zul', 'fra-wol']
maf_langs = ['en_lug', 'en_tsn', 'en_zul', 'fr_wol']

for hug_lang, maf_lang in zip(hug_langs, maf_langs):
  train_path = 'data/' + hug_lang + '/spc-' + maf_lang + '_train.tsv'
  dev_path = 'data/' + hug_lang + '/spc-' + maf_lang + '_dev.tsv'
  test_path = 'data/' + hug_lang + '/spc-' + maf_lang + '_test.tsv'

  data_to_classify = 'data/' + hug_lang + '/spc-' + maf_lang + '_to_classify.tsv'

  model_path = 'models/' + hug_lang
  if not os.path.exists(model_path):
    print("Creation of the " + hug_lang + " model folder...")
    os.makedirs(model_path)

  !python3 run-sp-class.py \
      --train \
      --eval=True \
      --model={model} \
      --model_path={model_path} \
      --train_data={train_path} \
      --val_data={dev_path} \
      --test_data={test_path} \
      --epochs=4
 
  val_loss = 100

  for x in os.listdir(model_path):
    if x.endswith(".pt"):
      if float(x[x.index('loss') + 5:x.index('loss') + 9]) < val_loss:
        val_loss = float(x[x.index('loss') + 5:x.index('loss') + 9])
        model_name = model_path + '/' + x

  output_path = 'data/' + hug_lang

  !python3 predict.py \
      --predict \
      --model={model} \
      --model_path={model_name} \
      --data_path={data_to_classify} \
      --output_path={output_path}

/content/drive/MyDrive/WMT22-MasaKhane/sentence-pair-classification
PyTorch version 1.11.0+cu113 available.
TensorFlow version 2.8.2 available.
Gen RAM Free: 87.4 GB  | Proc size: 798.8 MB
GPU RAM Free: 40536MB | Used: 0MB | Util   0% | Total 40536MB
Reading training data...
Downloading: 100% 684/684 [00:00<00:00, 612kB/s]
Downloading: 100% 760k/760k [00:00<00:00, 7.46MB/s]
Reading validation data...
Downloading: 100% 47.4M/47.4M [00:00<00:00, 68.3MB/s]
 20% 101/510 [00:06<00:23, 17.69it/s]
Iteration 102/510 of epoch 1 complete. Loss : 0.17076843892972843 
 40% 203/510 [00:11<00:16, 18.64it/s]
Iteration 204/510 of epoch 1 complete. Loss : 0.09903182465510041 
 60% 305/510 [00:17<00:11, 18.46it/s]
Iteration 306/510 of epoch 1 complete. Loss : 0.0787077126796266 
 80% 407/510 [00:22<00:05, 17.97it/s]
Iteration 408/510 of epoch 1 complete. Loss : 0.07042094746915002 
100% 509/510 [00:28<00:00, 18.48it/s]
Iteration 510/510 of epoch 1 complete. Loss : 0.07026049139790748 
100% 510/510 [00:2

In [ ]:
import pandas as pd
import os

%cd /content/drive/MyDrive/WMT22 MasaKhane/MasaKhaneNLP-WMT22/sentence-pair-classification

#hug_langs = ['eng-hau', 'eng-ibo', 'eng-lug', 'eng-swh', 'eng-tsn', 'eng-yor', 'eng-zul', 'fra-wol']
#maf_langs = ['en_hau', 'en_ibo', 'en_lug', 'en_swa', 'en_tsn', 'en_yor', 'en_zul', 'fr_wol']

hug_langs = ['eng-lug', 'eng-tsn', 'eng-zul', 'fra-wol']
maf_langs = ['en_lug', 'en_tsn', 'en_zul', 'fr_wol']

thresholds = [0.4, 0.5, 0.7]

for hug_lang, maf_lang in zip(hug_langs, maf_langs):

  with open('data/' + hug_lang + '/predictions.txt', 'r') as f:
      preds = f.readlines()

  df_pred = pd.read_csv('data/' + hug_lang + '/spc-' + maf_lang + '_to_classify.tsv', sep='\t')

  for threshold in thresholds:

    src_correct = []
    tgt_correct = []

    for sent1, sent2, pred in zip(df_pred['sentence1'], df_pred['sentence2'], preds):
      if float(pred) >= threshold:
        src_correct.append(sent1)
        tgt_correct.append(sent2)

    df = pd.DataFrame(list(zip(src_correct, tgt_correct)), columns=['input', 'target'])
    df.to_csv(os.path.join('data/' + hug_lang + '/', 'correct-translations_t_' + str(threshold) + '.tsv'), sep='\t', index=False)

    print(lang, threshold, len(df))